In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

In [ ]:
def VandermondeMatrix(x, y, n):
    X = np.c_[np.ones(len(x))]
    for i in range(1, n+1):
        # x-terms
        X = np.c_[X, x**(i)]
        # y-terms
        X = np.c_[X, y**(i)]
        # Cross terms
        for j in range(i-1, 0, -1):
            X = np.c_[X, (x**(j))*(y**(i-j))]
            
    return X

In [ ]:
def linreg_ols(X, z):
    
    # Solving for beta
    beta = np.linalg.inv(np.transpose(X).dot(X)).dot(np.transpose(X)).dot(z)
    
    y_predict_ols = X @ beta
    
    return beta, y_predict_ols

In [ ]:
def ridgereg(X, z, lambda_ridge):

    # Solving for beta
    beta_ridge = np.linalg.inv(np.transpose(X).dot(X) + lambda_ridge*np.identity(X.shape[1])).dot(np.transpose(X)).dot(z)

    y_predict_ridge = X @ beta_ridge
    
    return beta_ridge, y_predict_ridge

In [ ]:
from sklearn.linear_model import Lasso

def lassoreg(X, z, lambda_lasso):
    
    las = Lasso(alpha=lambda_lasso, fit_intercept = False)
    las.fit(X, z)
    
    beta = las.coef_#[:,np.newaxis]
    
    y_predict_lasso = X @ beta
    
    R2_lasso = las.score(X, z)
    
    return beta, y_predict_lasso, R2_lasso

In [ ]:
def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)

In [ ]:
def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

In [ ]:
# GENERATING DATA
x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)

#x = np.arange(0, 1, 0.009)
#y = np.arange(0, 1, 0.009)


x, y = np.meshgrid(x, y)

z = FrankeFunction(x, y)

x = np.ravel(x)
y = np.ravel(y)
z = np.ravel(z)

#z = z + 0.1 * np.random.randn(z.shape[0]) # noise level = 0.1
X = VandermondeMatrix(x, y, 5)

In [ ]:
beta_lasso, z_predict_lasso, R2_lasso = lassoreg(X, z, 0.001)
    
print(MSE(z, z_predict_lasso))
print(R2_lasso)

In [ ]:
%matplotlib inline

x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)

x, y = np.meshgrid(x, y)

z = FrankeFunction(x, y)

x = np.ravel(x)
y = np.ravel(y)
z = np.ravel(z)

X = VandermondeMatrix(x, y, 5)

#noise = [0, 0.1, 0.2, 0.3, 0.4]
noise = np.arange(0,0.4,0.01)
#noise = np.arange(0, 1, 0.001)

R2_scores = list()
MSE_scores = list()
R2_scores_ridge = list()
MSE_scores_ridge = list()
R2_scores_lasso = list()
MSE_scores_lasso = list()
    
for N in noise:
    z = z + N * np.random.randn(z.shape[0])

    beta_ols, z_predict_ols = linreg_ols(X, z)
    
    R2_scores.append(R2(z, z_predict_ols))
    MSE_scores.append(MSE(z, z_predict_ols))

    beta_ridge, z_predict_ridge = ridgereg(X, z, 0.001)
    
    R2_scores_ridge.append(R2(z, z_predict_ridge))
    MSE_scores_ridge.append(MSE(z, z_predict_ridge))
    
    beta_lasso, z_predict_lasso, R2_lasso = lassoreg(X, z, 0.001)
    
    R2_scores_lasso.append(R2_lasso)
    MSE_scores_lasso.append(MSE(z, z_predict_lasso))
    
    # Regenerate z
    z = 0
    x = np.arange(0, 1, 0.05)
    y = np.arange(0, 1, 0.05)

    x, y = np.meshgrid(x, y)

    z = FrankeFunction(x, y)
    z = np.ravel(z)
    
    
fig, ax1 = plt.subplots()

color = 'tab:cyan'
ax1.set_xlabel('Noise')
ax1.set_ylabel('R2', color=color)
ax1.plot(noise, R2_scores, color=color, ls = '-', label = 'OLS')
ax1.plot(noise, R2_scores_ridge, color=color, ls = ':', label = 'Ridge')
ax1.plot(noise, R2_scores_lasso, color=color, ls = '-.', label = 'Lasso')
ax1.tick_params(axis='y', labelcolor=color)

plt.legend()

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:orange'
ax2.set_ylabel('MSE', color=color)  # we already handled the x-label with ax1
ax2.plot(noise, MSE_scores, color=color, ls = '-', label = 'OLS')
ax2.plot(noise, MSE_scores_ridge, color=color, ls = ':', label = 'Ridge')
ax2.plot(noise, MSE_scores_lasso, color=color, ls = '-.', label = 'Lasso')
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

#ax1.legend(loc='best')

plt.legend()
plt.title('MSE and R2 as a function of noise, $n$ = 5')
plt.grid(axis='both')
plt.show()

In [ ]:
np.mean(MSE_scores)

In [ ]:
X = VandermondeMatrix(x, y, 5)

#lambdas = [0.1, 0.01, 0.001, 0.0001]
lambdas = np.arange(0,0.1,0.0001)

MSE_scores_ridge = list()
MSE_scores_lasso = list()
R2_scores_ridge = list()
R2_scores_lasso = list()
for la in lambdas:
    beta_ridge, z_predict_ridge = ridgereg(X, z, lambda_ridge = la)
    beta_lasso, z_predict_lasso, R2_lasso = lassoreg(X, z, lambda_lasso = la)
    MSE_scores_ridge.append(MSE(z, z_predict_ridge))
    MSE_scores_lasso.append(MSE(z, z_predict_lasso))
    R2_scores_ridge.append(R2(z, z_predict_ridge))
    R2_scores_lasso.append(R2_lasso)
        
#plt.plot(lambdas, MSE_scores_ridge, label='Ridge')
#plt.xlabel('Lambda')
#plt.ylabel('MSE Score')
#plt.plot(lambdas, MSE_scores_lasso, label='Lasso')
#plt.show()
"""
# PLOT OF LAMBDAS FOR RIDGE REGRESSION
fig, ax1 = plt.subplots()

color = 'tab:cyan'
ax1.set_xlabel('Lambdas')
ax1.set_ylabel('R2', color=color)
ax1.plot(lambdas, R2_scores_ridge, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:orange'
ax2.set_ylabel('MSE', color=color)  # we already handled the x-label with ax1
ax2.plot(lambdas, MSE_scores_ridge, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.title('Ridge regression (5-th order polynomial)')
plt.grid()
plt.show() 

"""

# PLOT OF LAMBDAS FOR LASSO REGRESSION
fig, ax1 = plt.subplots()

color = 'tab:cyan'
ax1.set_xlabel('Lambdas')
ax1.set_ylabel('R2', color=color)
ax1.plot(lambdas, R2_scores_ridge, color=color, ls=':', label = 'Ridge')
ax1.plot(lambdas, R2_scores_lasso, color=color, ls = '-.', label = 'Lasso')
ax1.tick_params(axis='y', labelcolor=color)
plt.legend()

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:orange'
ax2.set_ylabel('MSE', color=color)  # we already handled the x-label with ax1
ax2.plot(lambdas, MSE_scores_ridge, color=color, ls = ':', label= 'Ridge')
ax2.plot(lambdas, MSE_scores_lasso, color=color, ls = '-.', label = 'Lasso')
ax2.tick_params(axis='y', labelcolor=color)
plt.legend()

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.title('Variation of lambdas (5-th order polynomial)')
plt.grid()
plt.show()

In [ ]:
R2_scores = list()
MSE_scores = list()
R2_scores_ridge = list()
MSE_scores_ridge = list()
R2_scores_lasso = list()
MSE_scores_lasso = list()

for n in range(1,6):
    X = VandermondeMatrix(x, y, n)

    beta_ols, z_predict_ols = linreg_ols(X, z)    
    beta_ridge, z_predict_ridge = ridgereg(X, z, 0.001)
    beta_lasso, z_predict_lasso, R2_lasso = lassoreg(X, z, 0.001)
    
    R2_scores.append(R2(z, z_predict_ols))
    MSE_scores.append(MSE(z, z_predict_ols))
    R2_scores_ridge.append(R2(z, z_predict_ridge))
    MSE_scores_ridge.append(MSE(z, z_predict_ridge))
    R2_scores_lasso.append(R2_lasso)
    MSE_scores_lasso.append(MSE(z, z_predict_lasso))

#    lambdas = [0.1, 0.01, 0.001, 0.0001]
    
#    for la in lambdas:
#        beta_ridge, z_predict_ridge = ridgereg(X, z, lambda_ridge = la)
#        beta_lasso, z_predict_lasso, R2_lasso = lassoreg(X, z, lambda_lasso = la)

n = range(1,6)
fig, ax1 = plt.subplots()

color = 'tab:cyan'
ax1.set_xlabel('Model complexity (no noise)')
ax1.set_ylabel('R2', color=color)
ax1.plot(n, R2_scores, color=color, ls = '-', label = 'OLS')
ax1.plot(n, R2_scores_ridge, color=color, ls = ':', label = 'Ridge')
ax1.plot(n, R2_scores_lasso, color=color, ls = '-.', label = 'Lasso')
ax1.tick_params(axis='y', labelcolor=color)
plt.legend()

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:orange'
ax2.set_ylabel('MSE', color=color)  # we already handled the x-label with ax1
ax2.plot(n, MSE_scores, color=color, ls = '-', label = 'OLS')
ax2.plot(n, MSE_scores_ridge, color=color, ls = ':', label = 'Ridge')
ax2.plot(n, MSE_scores_lasso, color=color, ls = '-.', label = 'Lasso')
ax2.tick_params(axis='y', labelcolor=color)
plt.legend()

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.title('Ordinary Least Squares')
plt.grid()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

R2_scores = list()
MSE_scores_train = list()
MSE_scores_test = list()
for n in range(1,10):
    X = VandermondeMatrix(x, y, n)
    
    #np.random.shuffle(X)
    
    X_train, X_test, z_train, z_test = train_test_split(X, z, test_size = 0.2)
    
    beta_ols_train, z_predict_ols_train = linreg_ols(X_train, z_train)
    z_predict_ols_test = X_test @ beta_ols_train
    
    MSE_scores_train.append(MSE(z_train, z_predict_ols_train))
    MSE_scores_test.append(MSE(z_test, z_predict_ols_test))
    
n = range(1,10)

plt.plot(n, MSE_scores_train, label='Train')
plt.plot(n, MSE_scores_test, label='Test')
plt.xlabel('Model complexity')
plt.ylabel('MSE')
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random


fig = pyplot.figure()
ax = Axes3D(fig)

ax.scatter(x, y, z_predict_ols)

In [ ]:
# The Franke function (code provided in the assignment text)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed

def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)

x, y = np.meshgrid(x, y)

z = FrankeFunction(x, y)

x = np.ravel(x)
y = np.ravel(y)
z = np.ravel(z)

#z = z + 0.1 * np.random.randn(z.shape[0]) # noise level = 0.1
X = VandermondeMatrix(x, y, 5)

beta_ols, z_predict_ols = linreg_ols(X, z)

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = fig.gca(projection='3d')



x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)

x, y = np.meshgrid(x, y)

z = FrankeFunction(x, y)


# Plot the surface
#surf = ax.plot_surface(x, y, z, cmap=cm.coolwarm, linewidth=0, antialiased=False)

ax.scatter(x, y, z_predict_ols)

# Customize the z axis
#ax.set_zlim(-0.10, 1.40)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
# GENERATING DATA
x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)

#x = np.arange(0, 1, 0.009)
#y = np.arange(0, 1, 0.009)


x, y = np.meshgrid(x, y)

z = FrankeFunction(x, y)

x = np.ravel(x)
y = np.ravel(y)
z = np.ravel(z)


z = z + 0.1 * np.random.randn(z.shape[0]) # noise level = 0.1

In [ ]:
MSE_scores_ridge = list()
MSE_scores_lasso = list()
R2_scores_ridge = list()
R2_scores_lasso = list()
for n in range(1,6):

    X = VandermondeMatrix(x, y, n)

    lambdas = np.arange(0,0.1,0.0001)

    for la in lambdas:
        beta_ridge, z_predict_ridge = ridgereg(X, z, lambda_ridge = la)
        beta_lasso, z_predict_lasso, R2_lasso = lassoreg(X, z, lambda_lasso = la)
        MSE_scores_ridge.append(MSE(z, z_predict_ridge))
        MSE_scores_lasso.append(MSE(z, z_predict_lasso))
        R2_scores_ridge.append(R2(z, z_predict_ridge))
        R2_scores_lasso.append(R2_lasso)
    
    print(n)
    print(len(R2_scores_ridge))
    
""" 
# PLOT OF LAMBDAS FOR RIDGE REGRESSION
    fig, ax1 = plt.subplots()

    color = 'tab:cyan'
    ax1.set_xlabel('Lambdas')
    ax1.set_ylabel('R2', color=color)
    ax1.plot(lambdas, R2_scores_ridge, color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:orange'
    ax2.set_ylabel('MSE', color=color)  # we already handled the x-label with ax1
    ax2.plot(lambdas, MSE_scores_ridge, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped

    plt.title('Ridge regression (3rd order polynomial)')
    plt.grid()
    plt.show()
"""
    

In [ ]:
# PLOT OF LAMBDAS FOR RIDGE REGRESSION
fig, ax1 = plt.subplots()

color = 'tab:cyan'
ax1.set_xlabel('Lambdas')
ax1.set_ylabel('R2', color=color)
ax1.plot(lambdas, R2_scores_ridge[0:1000], color=color, label = 'n = 1')
ax1.plot(lambdas, R2_scores_ridge[1000:2000], color=color, linestyle = ':', label = 'n = 2')
ax1.plot(lambdas, R2_scores_ridge[2000:3000], color=color, linestyle = '-.', label = 'n = 3')
ax1.plot(lambdas, R2_scores_ridge[3000:4000], color=color, linestyle = '--', label = 'n = 4')
ax1.plot(lambdas, R2_scores_ridge[4000:5000], label = 'n = 5')
ax1.tick_params(axis='y', labelcolor=color)
plt.legend()

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:orange'
ax2.set_ylabel('MSE', color=color)  # we already handled the x-label with ax1
ax2.plot(lambdas, MSE_scores_ridge[0:1000], color=color, label = 'MSE n = 1')
ax2.plot(lambdas, MSE_scores_ridge[1000:2000], color=color, linestyle = ':', label = 'MSE n = 2')
ax2.plot(lambdas, MSE_scores_ridge[2000:3000], color=color, linestyle = '-.', label = 'MSE n = 3')
ax2.plot(lambdas, MSE_scores_ridge[3000:4000], color=color, linestyle = '--', label = 'MSE n = 4')
ax2.plot(lambdas, MSE_scores_ridge[4000:5000], label = 'MSE n = 5')
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.title('Ridge regression (3rd order polynomial)')
plt.legend()
plt.grid()
plt.show()